In [1]:
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
import math as math
import helpers
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
players = pd.read_csv(r'C:\Shubhanshu\NEU\SML\Project\player_data_complete.csv')
players

,Unnamed: 0,Unnamed: 0.1,Year,Name,Positions,Age,Rating,Potential,Team,Contract,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Joined,Loan End Date,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,HITS
0,0,0,2008,Ronaldinho,CAM,27,91,93,FC Barcelona,2010,28130,"5'11""",179lbs,Right,91,CAM,2,NaN,NaN,€0,€0,€0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4 ★,1★,NaN,NaN,5 ★,90,89,87,96,41,75,61
1,1,1,2008,Cristiano Ronaldo,RW,22,91,94,Manchester United,2012,20801,"6'1""",183lbs,Right,91,CF,3,NaN,NaN,€0,€0,€0,338,88,85,84,81,NaN,353,97,NaN,88,73,95,275,95,94,NaN,86,NaN,340,90,NaN,91,84,75,283,61,69,74,NaN,79,NaN,54,27,27,NaN,72,9,21,73,21,21,1715,476,4 ★,1★,NaN,NaN,5 ★,94,83,81,95,41,82,403
2,2,2,2008,T. Henry,ST,29,91,91,FC Barcelona,2011,1625,"6'2""",183lbs,Right,91,ST,0,NaN,NaN,€0,€0,€0,322,76,95,65,86,NaN,339,92,NaN,89,67,91,279,94,93,NaN,92,NaN,333,86,NaN,83,78,86,304,40,83,89,NaN,92,NaN,42,21,21,NaN,77,14,21,67,21,21,1696,465,4 ★,1★,NaN,NaN,5 ★,93,91,79,92,38,72,18
3,3,3,2008,G. Buffon,GK,29,91,93,Juventus,2012,1179,"6'3""",181lbs,Right,91,GK,2,NaN,NaN,€0,€0,€0,86,21,21,21,23,NaN,131,21,NaN,9,79,22,195,62,44,NaN,89,NaN,182,24,NaN,56,81,21,273,81,90,14,NaN,88,NaN,42,21,21,NaN,367,90,91,79,96,90,1276,498,3 ★,1★,NaN,NaN,5 ★,90,91,79,90,52,96,85
4,4,4,2008,A. Nesta,CB,31,91,92,Milan,2011,1088,"6'2""",174lbs,Right,91,CB,1,NaN,NaN,€0,€0,€0,213,42,24,93,54,NaN,180,52,NaN,25,34,69,229,73,78,NaN,78,NaN,282,65,NaN,88,90,39,361,88,92,89,NaN,92,NaN,188,93,95,NaN,70,7,21,34,21,21,1523,407,3 ★,1★,NaN,NaN,4 ★,76,43,43,62,94,89,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,233658,17995,2020,Du Wenyang,"RB,CB",29,56,56,Beijing Renhe FC,2019 ~ 2021,231962,"6'0""",172lbs,Right,56,RB,0,"Feb 18, 2019",NaN,€70K,€2K,€109K,248,55,52,48,50,43.0,250,54,46.0,47,52,51,296,64,62,62.0,44,64.0,301,63,55.0,70,62,51,292,72,58,53,58.0,51,51.0,162,51,57,54.0,45,5,7,10,15,8,1594,344,2 ★,2★,Medium,Medium,1 ★,63,54,53,54,54,66,1
233659,233659,17996,2020,Kim Ju Sung,"CB,CDM",18,56,76,FC Seoul,2019 ~ 2023,251162,"6'1""",168lbs,Left,57,CB,20,"Jan 4, 2019",NaN,€170K,€500,€383K,173,23,20,54,47,29.0,177,42,22.0,23,46,44,277,65,64,48.0,46,54.0,251,37,72.0,54,64,24,195,40,56,25,35.0,39,44.0,170,55,58,57.0,56,12,14,9,11,10,1299,284,3 ★,2★,Low,High,1 ★,64,26,37,44,56,57,1
233660,233660,17997,2020,D. Miljanović,LM,18,56,68,AFC Eskilstuna,2018 ~ 2022,247835,"5'10""",165lbs,Right,57,RM,12,"Apr 4, 2018",NaN,€150K,€500,€218K,246,57,51,35,58,45.0,252,54,50.0,36,55,57,290,65,69,51.0,38,67.0,235,55,47.0,36,55,42,223,36,33,54,54.0,46,51.0,71,21,25,25.0,38,7,10,6,10,5,1355,298,3 ★,2★,Medium,Medium,1 ★,67,50,55,54,26,46,1
233661,233661,17998,2020,J. Semple,"ST,RW,LW",18,56,78,Motherwell,2018 ~ 2021,248347,"6'0""",172lbs,Right,57,ST,22,"Jul 1, 2018",NaN,€210K,€600,€578K,249,48,56,50,52,43.0,228,50,50.0,40,35,53,318,77,75,60.0,41,65.0,276,73,51.0,55,49,48,205,24,26,52,50.0,53,52.0,66,20,26,20.0,59,9,11,10,14,15,1401,305,3 ★,2★,High,Medium,1 ★,76,57,48,52,26,46,3


### Data Cleaning and Feature Engineering for Players Table

In [4]:
# Dropping unnecessary columns
players.drop(labels = ['Unnamed: 0','Unnamed: 0.1','HITS'],axis=1,inplace = True)
# Creating total positions a player can play
players['TotalPositions'] = players.Positions.str.split(',', expand=False).agg([len])

In [5]:
# converting Height to cm
players["Height"] = players["Height"].apply(lambda x:helpers.change_to_cm(x))

In [6]:
# Deleting LBS from weight column
players["Weight"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
# Removing star from weak foot,skill moves and international reputation
players["Weak Foot"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
players["Skill Moves"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
players["International Reputation"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [7]:
# Splitting positions
players = pd.concat([players, players.Positions.str.split(',', expand=True)], axis=1)

players.drop('Positions',axis=1,inplace = True)  # Dropping the original column

# Renaming the position columns
players.rename(columns = {0 : 'Position 1', 1 : 'Position 2', 2: 'Position 3', 3: 'Position 4'}, inplace = True)

In [8]:
# Splitting Contract feature
players = pd.concat([players, players.Contract.str.split('~', expand=True)], axis=1)
players.drop('Contract',axis=1,inplace = True) 
players.rename(columns = {0 : 'Contract Start Date', 1 : 'Contract End Date'}, inplace = True)
players

,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Joined,Loan End Date,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date
0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,NaN,NaN,€0,€0,€0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,None,None,None,2010,None
1,2008,Cristiano Ronaldo,22,91,94,Manchester United,20801,185.4,183,Right,91,CF,3,NaN,NaN,€0,€0,€0,338,88,85,84,81,NaN,353,97,NaN,88,73,95,275,95,94,NaN,86,NaN,340,90,NaN,91,84,75,283,61,69,74,NaN,79,NaN,54,27,27,NaN,72,9,21,73,21,21,1715,476,4,1,NaN,NaN,5,94,83,81,95,41,82,1,RW,None,None,None,2012,None
2,2008,T. Henry,29,91,91,FC Barcelona,1625,188.0,183,Right,91,ST,0,NaN,NaN,€0,€0,€0,322,76,95,65,86,NaN,339,92,NaN,89,67,91,279,94,93,NaN,92,NaN,333,86,NaN,83,78,86,304,40,83,89,NaN,92,NaN,42,21,21,NaN,77,14,21,67,21,21,1696,465,4,1,NaN,NaN,5,93,91,79,92,38,72,1,ST,None,None,None,2011,None
3,2008,G. Buffon,29,91,93,Juventus,1179,190.5,181,Right,91,GK,2,NaN,NaN,€0,€0,€0,86,21,21,21,23,NaN,131,21,NaN,9,79,22,195,62,44,NaN,89,NaN,182,24,NaN,56,81,21,273,81,90,14,NaN,88,NaN,42,21,21,NaN,367,90,91,79,96,90,1276,498,3,1,NaN,NaN,5,90,91,79,90,52,96,1,GK,None,None,None,2012,None
4,2008,A. Nesta,31,91,92,Milan,1088,188.0,174,Right,91,CB,1,NaN,NaN,€0,€0,€0,213,42,24,93,54,NaN,180,52,NaN,25,34,69,229,73,78,NaN,78,NaN,282,65,NaN,88,90,39,361,88,92,89,NaN,92,NaN,188,93,95,NaN,70,7,21,34,21,21,1523,407,3,1,NaN,NaN,4,76,43,43,62,94,89,1,CB,None,None,None,2011,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,2020,Du Wenyang,29,56,56,Beijing Renhe FC,231962,182.9,172,Right,56,RB,0,"Feb 18, 2019",NaN,€70K,€2K,€109K,248,55,52,48,50,43.0,250,54,46.0,47,52,51,296,64,62,62.0,44,64.0,301,63,55.0,70,62,51,292,72,58,53,58.0,51,51.0,162,51,57,54.0,45,5,7,10,15,8,1594,344,2,2,Medium,Medium,1,63,54,53,54,54,66,2,RB,CB,None,None,2019,2021
233659,2020,Kim Ju Sung,18,56,76,FC Seoul,251162,185.4,168,Left,57,CB,20,"Jan 4, 2019",NaN,€170K,€500,€383K,173,23,20,54,47,29.0,177,42,22.0,23,46,44,277,65,64,48.0,46,54.0,251,37,72.0,54,64,24,195,40,56,25,35.0,39,44.0,170,55,58,57.0,56,12,14,9,11,10,1299,284,3,2,Low,High,1,64,26,37,44,56,57,2,CB,CDM,None,None,2019,2023
233660,2020,D. Miljanović,18,56,68,AFC Eskilstuna,247835,177.8,165,Right,57,RM,12,"Apr 4, 2018",NaN,€150K,€500,€218K,246,57,51,35,58,45.0,252,54,50.0,36,55,57,290,65,69,51.0,38,67.0,235,55,47.0,36,55,42,223,36,33,54,54.0,46,51.0,71,21,25,25.0,38,7,10,6,10,5,1355,298,3,2,Medium,Medium,1,67,50,55,54,26,46,1,LM,None,None,None,2018,2022
233661,2020,J. Semple,18,56,78,Motherwell,248347,182.9,172,Right,57,ST,22,"Jul 1, 2018",NaN,€210K,€600,€578K,249,48,56,50,52,43.0,228,50,50.0,40,35,53,318,77,75,60.0,41,65.0,276,73,51.0,55,49,48,205,24,26,52,50.0,53,52.0,66,20,26,20.0,59,9,11,10,14,15,1401,305,3,2,High,Medium,1,76,57,48,52,26,46,3,ST,RW,LW,None,2018,2021


In [12]:
players['Contract End Date'] = players[['Contract Start Date','Contract End Date']].apply(lambda x: helpers.contract_end_date(x[0], x[1]), axis=1)

In [13]:
players

,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Joined,Loan End Date,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date
0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,NaN,NaN,€0,€0,€0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,None,None,None,2010,2010
1,2008,Cristiano Ronaldo,22,91,94,Manchester United,20801,185.4,183,Right,91,CF,3,NaN,NaN,€0,€0,€0,338,88,85,84,81,NaN,353,97,NaN,88,73,95,275,95,94,NaN,86,NaN,340,90,NaN,91,84,75,283,61,69,74,NaN,79,NaN,54,27,27,NaN,72,9,21,73,21,21,1715,476,4,1,NaN,NaN,5,94,83,81,95,41,82,1,RW,None,None,None,2012,2012
2,2008,T. Henry,29,91,91,FC Barcelona,1625,188.0,183,Right,91,ST,0,NaN,NaN,€0,€0,€0,322,76,95,65,86,NaN,339,92,NaN,89,67,91,279,94,93,NaN,92,NaN,333,86,NaN,83,78,86,304,40,83,89,NaN,92,NaN,42,21,21,NaN,77,14,21,67,21,21,1696,465,4,1,NaN,NaN,5,93,91,79,92,38,72,1,ST,None,None,None,2011,2011
3,2008,G. Buffon,29,91,93,Juventus,1179,190.5,181,Right,91,GK,2,NaN,NaN,€0,€0,€0,86,21,21,21,23,NaN,131,21,NaN,9,79,22,195,62,44,NaN,89,NaN,182,24,NaN,56,81,21,273,81,90,14,NaN,88,NaN,42,21,21,NaN,367,90,91,79,96,90,1276,498,3,1,NaN,NaN,5,90,91,79,90,52,96,1,GK,None,None,None,2012,2012
4,2008,A. Nesta,31,91,92,Milan,1088,188.0,174,Right,91,CB,1,NaN,NaN,€0,€0,€0,213,42,24,93,54,NaN,180,52,NaN,25,34,69,229,73,78,NaN,78,NaN,282,65,NaN,88,90,39,361,88,92,89,NaN,92,NaN,188,93,95,NaN,70,7,21,34,21,21,1523,407,3,1,NaN,NaN,4,76,43,43,62,94,89,1,CB,None,None,None,2011,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,2020,Du Wenyang,29,56,56,Beijing Renhe FC,231962,182.9,172,Right,56,RB,0,"Feb 18, 2019",NaN,€70K,€2K,€109K,248,55,52,48,50,43.0,250,54,46.0,47,52,51,296,64,62,62.0,44,64.0,301,63,55.0,70,62,51,292,72,58,53,58.0,51,51.0,162,51,57,54.0,45,5,7,10,15,8,1594,344,2,2,Medium,Medium,1,63,54,53,54,54,66,2,RB,CB,None,None,2019,2019
233659,2020,Kim Ju Sung,18,56,76,FC Seoul,251162,185.4,168,Left,57,CB,20,"Jan 4, 2019",NaN,€170K,€500,€383K,173,23,20,54,47,29.0,177,42,22.0,23,46,44,277,65,64,48.0,46,54.0,251,37,72.0,54,64,24,195,40,56,25,35.0,39,44.0,170,55,58,57.0,56,12,14,9,11,10,1299,284,3,2,Low,High,1,64,26,37,44,56,57,2,CB,CDM,None,None,2019,2019
233660,2020,D. Miljanović,18,56,68,AFC Eskilstuna,247835,177.8,165,Right,57,RM,12,"Apr 4, 2018",NaN,€150K,€500,€218K,246,57,51,35,58,45.0,252,54,50.0,36,55,57,290,65,69,51.0,38,67.0,235,55,47.0,36,55,42,223,36,33,54,54.0,46,51.0,71,21,25,25.0,38,7,10,6,10,5,1355,298,3,2,Medium,Medium,1,67,50,55,54,26,46,1,LM,None,None,None,2018,2018
233661,2020,J. Semple,18,56,78,Motherwell,248347,182.9,172,Right,57,ST,22,"Jul 1, 2018",NaN,€210K,€600,€578K,249,48,56,50,52,43.0,228,50,50.0,40,35,53,318,77,75,60.0,41,65.0,276,73,51.0,55,49,48,205,24,26,52,50.0,53,52.0,66,20,26,20.0,59,9,11,10,14,15,1401,305,3,2,High,Medium,1,76,57,48,52,26,46,3,ST,RW,LW,None,2018,2018


In [9]:
#players['Contract End Date'] = players.apply(lambda x: helpers.contract_end_date(x[-2], x[-1]), axis=1)
players['Contract End Date'] = players['Contract End Date'].str.strip()
players['Contract End Date'] = players['Contract End Date'].replace({'Free': '0'})
players['Contract End Date'] =  players['Contract End Date'].fillna('0')
players

,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Joined,Loan End Date,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date
0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,NaN,NaN,€0,€0,€0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,None,None,None,2010,2010
1,2008,Cristiano Ronaldo,22,91,94,Manchester United,20801,185.4,183,Right,91,CF,3,NaN,NaN,€0,€0,€0,338,88,85,84,81,NaN,353,97,NaN,88,73,95,275,95,94,NaN,86,NaN,340,90,NaN,91,84,75,283,61,69,74,NaN,79,NaN,54,27,27,NaN,72,9,21,73,21,21,1715,476,4,1,NaN,NaN,5,94,83,81,95,41,82,1,RW,None,None,None,2012,2012
2,2008,T. Henry,29,91,91,FC Barcelona,1625,188.0,183,Right,91,ST,0,NaN,NaN,€0,€0,€0,322,76,95,65,86,NaN,339,92,NaN,89,67,91,279,94,93,NaN,92,NaN,333,86,NaN,83,78,86,304,40,83,89,NaN,92,NaN,42,21,21,NaN,77,14,21,67,21,21,1696,465,4,1,NaN,NaN,5,93,91,79,92,38,72,1,ST,None,None,None,2011,2011
3,2008,G. Buffon,29,91,93,Juventus,1179,190.5,181,Right,91,GK,2,NaN,NaN,€0,€0,€0,86,21,21,21,23,NaN,131,21,NaN,9,79,22,195,62,44,NaN,89,NaN,182,24,NaN,56,81,21,273,81,90,14,NaN,88,NaN,42,21,21,NaN,367,90,91,79,96,90,1276,498,3,1,NaN,NaN,5,90,91,79,90,52,96,1,GK,None,None,None,2012,2012
4,2008,A. Nesta,31,91,92,Milan,1088,188.0,174,Right,91,CB,1,NaN,NaN,€0,€0,€0,213,42,24,93,54,NaN,180,52,NaN,25,34,69,229,73,78,NaN,78,NaN,282,65,NaN,88,90,39,361,88,92,89,NaN,92,NaN,188,93,95,NaN,70,7,21,34,21,21,1523,407,3,1,NaN,NaN,4,76,43,43,62,94,89,1,CB,None,None,None,2011,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,2020,Du Wenyang,29,56,56,Beijing Renhe FC,231962,182.9,172,Right,56,RB,0,"Feb 18, 2019",NaN,€70K,€2K,€109K,248,55,52,48,50,43.0,250,54,46.0,47,52,51,296,64,62,62.0,44,64.0,301,63,55.0,70,62,51,292,72,58,53,58.0,51,51.0,162,51,57,54.0,45,5,7,10,15,8,1594,344,2,2,Medium,Medium,1,63,54,53,54,54,66,2,RB,CB,None,None,2019,2021
233659,2020,Kim Ju Sung,18,56,76,FC Seoul,251162,185.4,168,Left,57,CB,20,"Jan 4, 2019",NaN,€170K,€500,€383K,173,23,20,54,47,29.0,177,42,22.0,23,46,44,277,65,64,48.0,46,54.0,251,37,72.0,54,64,24,195,40,56,25,35.0,39,44.0,170,55,58,57.0,56,12,14,9,11,10,1299,284,3,2,Low,High,1,64,26,37,44,56,57,2,CB,CDM,None,None,2019,2023
233660,2020,D. Miljanović,18,56,68,AFC Eskilstuna,247835,177.8,165,Right,57,RM,12,"Apr 4, 2018",NaN,€150K,€500,€218K,246,57,51,35,58,45.0,252,54,50.0,36,55,57,290,65,69,51.0,38,67.0,235,55,47.0,36,55,42,223,36,33,54,54.0,46,51.0,71,21,25,25.0,38,7,10,6,10,5,1355,298,3,2,Medium,Medium,1,67,50,55,54,26,46,1,LM,None,None,None,2018,2022
233661,2020,J. Semple,18,56,78,Motherwell,248347,182.9,172,Right,57,ST,22,"Jul 1, 2018",NaN,€210K,€600,€578K,249,48,56,50,52,43.0,228,50,50.0,40,35,53,318,77,75,60.0,41,65.0,276,73,51.0,55,49,48,205,24,26,52,50.0,53,52.0,66,20,26,20.0,59,9,11,10,14,15,1401,305,3,2,High,Medium,1,76,57,48,52,26,46,3,ST,RW,LW,None,2018,2021


In [10]:
players['Contract End Date'] = players.apply(lambda x: helpers.check_loan(x[-1]), axis=1)

In [11]:
players['Contract End Date'] = pd.to_numeric(players['Contract End Date'])

In [12]:
players['Contract Remaining'] = players.apply(lambda x: helpers.contract_remaining(x[-1], x[0]), axis=1)
players['Release Clause'] = players.apply(lambda x: helpers.player_value(x['Release Clause']), axis=1)
players['Value'] = players.apply(lambda x: helpers.player_value(x['Value']), axis=1)
players['Wage'] = players.apply(lambda x: helpers.player_value(x['Wage']), axis=1)
players

,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Joined,Loan End Date,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date,Contract Remaining
0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,NaN,NaN,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,None,None,None,2010,2010.0,2
1,2008,Cristiano Ronaldo,22,91,94,Manchester United,20801,185.4,183,Right,91,CF,3,NaN,NaN,0,0,0,338,88,85,84,81,NaN,353,97,NaN,88,73,95,275,95,94,NaN,86,NaN,340,90,NaN,91,84,75,283,61,69,74,NaN,79,NaN,54,27,27,NaN,72,9,21,73,21,21,1715,476,4,1,NaN,NaN,5,94,83,81,95,41,82,1,RW,None,None,None,2012,2012.0,4
2,2008,T. Henry,29,91,91,FC Barcelona,1625,188.0,183,Right,91,ST,0,NaN,NaN,0,0,0,322,76,95,65,86,NaN,339,92,NaN,89,67,91,279,94,93,NaN,92,NaN,333,86,NaN,83,78,86,304,40,83,89,NaN,92,NaN,42,21,21,NaN,77,14,21,67,21,21,1696,465,4,1,NaN,NaN,5,93,91,79,92,38,72,1,ST,None,None,None,2011,2011.0,3
3,2008,G. Buffon,29,91,93,Juventus,1179,190.5,181,Right,91,GK,2,NaN,NaN,0,0,0,86,21,21,21,23,NaN,131,21,NaN,9,79,22,195,62,44,NaN,89,NaN,182,24,NaN,56,81,21,273,81,90,14,NaN,88,NaN,42,21,21,NaN,367,90,91,79,96,90,1276,498,3,1,NaN,NaN,5,90,91,79,90,52,96,1,GK,None,None,None,2012,2012.0,4
4,2008,A. Nesta,31,91,92,Milan,1088,188.0,174,Right,91,CB,1,NaN,NaN,0,0,0,213,42,24,93,54,NaN,180,52,NaN,25,34,69,229,73,78,NaN,78,NaN,282,65,NaN,88,90,39,361,88,92,89,NaN,92,NaN,188,93,95,NaN,70,7,21,34,21,21,1523,407,3,1,NaN,NaN,4,76,43,43,62,94,89,1,CB,None,None,None,2011,2011.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,2020,Du Wenyang,29,56,56,Beijing Renhe FC,231962,182.9,172,Right,56,RB,0,"Feb 18, 2019",NaN,70000,2000,109000,248,55,52,48,50,43.0,250,54,46.0,47,52,51,296,64,62,62.0,44,64.0,301,63,55.0,70,62,51,292,72,58,53,58.0,51,51.0,162,51,57,54.0,45,5,7,10,15,8,1594,344,2,2,Medium,Medium,1,63,54,53,54,54,66,2,RB,CB,None,None,2019,2021.0,1
233659,2020,Kim Ju Sung,18,56,76,FC Seoul,251162,185.4,168,Left,57,CB,20,"Jan 4, 2019",NaN,170000,500,383000,173,23,20,54,47,29.0,177,42,22.0,23,46,44,277,65,64,48.0,46,54.0,251,37,72.0,54,64,24,195,40,56,25,35.0,39,44.0,170,55,58,57.0,56,12,14,9,11,10,1299,284,3,2,Low,High,1,64,26,37,44,56,57,2,CB,CDM,None,None,2019,2023.0,3
233660,2020,D. Miljanović,18,56,68,AFC Eskilstuna,247835,177.8,165,Right,57,RM,12,"Apr 4, 2018",NaN,150000,500,218000,246,57,51,35,58,45.0,252,54,50.0,36,55,57,290,65,69,51.0,38,67.0,235,55,47.0,36,55,42,223,36,33,54,54.0,46,51.0,71,21,25,25.0,38,7,10,6,10,5,1355,298,3,2,Medium,Medium,1,67,50,55,54,26,46,1,LM,None,None,None,2018,2022.0,2
233661,2020,J. Semple,18,56,78,Motherwell,248347,182.9,172,Right,57,ST,22,"Jul 1, 2018",NaN,210000,600,578000,249,48,56,50,52,43.0,228,50,50.0,40,35,53,318,77,75,60.0,41,65.0,276,73,51.0,55,49,48,205,24,26,52,50.0,53,52.0,66,20,26,20.0,59,9,11,10,14,15,1401,305,3,2,High,Medium,1,76,57,48,52,26,46,3,ST,RW,LW,None,2018,2021.0,1


#### Checking for Missing Values and DataTypes

In [13]:
# Checking missing values in the dataset
missing_Values_Columns = players.isnull().sum(axis = 0)
missing_Values_Percentage = missing_Values_Columns * 100/players.shape[0]
missing_Value_df = missing_Values_Percentage.to_frame()
missing_Value_df[missing_Value_df[0] > 0]

,0
Team,2.684636
Joined,23.053714
Loan End Date,96.931050
Volleys,23.053714
Curve,23.053714
Agility,23.053714
Balance,23.053714
Jumping,23.053714
Vision,23.053714
Composure,69.211214


In [14]:
# Already taken care in Contract start and end date
players.drop(['Joined','Loan End Date'],axis=1,inplace = True) 

In [15]:
# Since these players are free agents, adding No Team in their Teams Coloumn
players['Team'] = players['Team'].fillna('No Team')

In [16]:
# Changinng object columns to category columns

changeTypeToCategory = ['Attacking Work Rate','Defensive Work Rate']
for col in changeTypeToCategory:
    players[col] = players[col].astype('category')

In [17]:
changeTypeToInt = ['Weight','Weak Foot','Skill Moves','International Reputation']
for col in changeTypeToInt:
    players[col] = players[col].astype('int64')

In [18]:
# Getting the positions on a football pitch
footballPositions = pd.read_csv(r'C:\Shubhanshu\NEU\SML\Project\PlayerPositions.csv')
footballPositions

,Position,Category
0,GK,GoalKeeper
1,ST,Forward
2,CB,Defender
3,CAM,Midfielder
4,CDM,Midfielder
5,CF,Forward
6,RM,Midfielder
7,CM,Midfielder
8,RW,Forward
9,RB,Defender


In [19]:
# Joining players and positions dataset
players_df = pd.merge(players, footballPositions, left_on='Best Position', right_on='Position')
players_df.drop(['Position'],axis=1,inplace = True)

In [20]:
players_df

,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date,Contract Remaining,Category
0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,None,None,None,2010,2010.0,2,Midfielder
1,2008,Kaká,25,90,91,Milan,138449,182.9,161,Right,90,CAM,1,0,0,0,334,87,90,64,93,NaN,353,91,NaN,80,90,92,271,93,89,NaN,89,NaN,330,86,NaN,78,73,93,288,38,87,86,NaN,77,NaN,51,22,29,NaN,70,4,22,90,22,22,1697,470,4,1,NaN,NaN,4,91,89,89,91,42,68,1,CAM,None,None,None,2009,2009.0,1,Midfielder
2,2008,Diego,22,88,90,SV Werder Bremen,136138,172.7,161,Right,88,CAM,2,0,0,0,316,84,85,55,92,NaN,350,90,NaN,83,84,93,252,86,83,NaN,83,NaN,301,82,NaN,78,62,79,298,64,80,83,NaN,71,NaN,66,25,41,NaN,75,9,22,84,22,22,1658,455,4,1,NaN,NaN,3,84,82,87,90,45,67,1,CAM,None,None,None,2010,2010.0,2,Midfielder
3,2008,S. Nasri,20,85,90,Olympique de Marseille,165239,177.8,165,Right,85,CAM,5,0,0,0,310,77,76,67,90,NaN,334,88,NaN,75,84,87,247,82,81,NaN,84,NaN,299,76,NaN,75,69,79,300,64,89,80,NaN,67,NaN,84,37,47,NaN,75,12,21,84,21,21,1649,453,4,1,NaN,NaN,3,81,77,84,87,54,70,1,CAM,None,None,None,2009,2009.0,1,Midfielder
4,2008,Deco,29,85,90,FC Barcelona,53769,177.8,161,Right,85,CAM,5,0,0,0,305,82,74,58,91,NaN,342,82,NaN,84,86,90,254,86,81,NaN,87,NaN,314,74,NaN,82,72,86,332,71,89,84,NaN,88,NaN,47,25,22,NaN,75,12,21,86,21,21,1669,449,4,1,NaN,NaN,4,83,78,87,86,40,75,1,CAM,None,None,None,2010,2010.0,2,Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,2020,L. Klassen,19,58,75,TSV 1860 München,244181,172.7,152,Left,60,LWB,17,250000,500,519000,225,59,39,39,59,29.0,241,59,39.0,31,49,63,349,75,71,73.0,49,81.0,262,40,72.0,67,51,32,220,57,49,41,41.0,32,52.0,156,50,51,55.0,46,10,9,6,10,11,1499,331,3,2,High,Medium,1,73,37,52,62,50,57,2,LB,LM,None,None,2018,2020.0,0,Defender
233659,2020,J. King,18,58,75,Sydney FC,246005,177.8,150,Left,60,LWB,17,240000,500,450000,202,52,27,39,56,28.0,217,56,26.0,30,50,55,331,76,71,65.0,50,69.0,255,29,65.0,76,56,29,230,52,54,55,38.0,31,41.0,163,55,56,52.0,50,8,10,11,11,10,1448,321,3,2,Medium,Medium,1,73,29,48,57,53,61,1,LB,None,None,None,2018,2020.0,0,Defender
233660,2020,L. Obexer,22,57,68,FC Lugano,233502,177.8,159,Left,58,LWB,11,160000,1000,0,218,54,36,50,47,31.0,214,59,36.0,30,39,50,323,68,66,58.0,55,76.0,250,34,62.0,65,60,29,238,55,52,51,40.0,40,41.0,166,52,58,56.0,45,14,12,8,5,6,1454,317,3,2,Medium,Medium,1,67,35,44,57,54,60,2,LWB,LB,None,None,"Jun 30, 2020 On Loan",2020.0,0,Defender
233661,2020,R. Donelon,23,57,62,Sligo Rovers,223091,170.2,157,Left,59,LWB,5,120000,500,195000,197,58,22,42,56,19.0,291,57,59.0,66,54,55,321,63,66,68.0,45,79.0,291,45,66.0,74,56,50,246,49,52,58,52.0,35,57.0,164,55,53,56.0,63,14,14,12,9,14,1573,327,2,2,High,Medium,1,65,35,56,58,53,60,3,LB,LWB,LM,None,2013,2020.0,0,Defender


In [21]:
players_df.to_csv(f'player_data_selected.csv', encoding='utf-8-sig')

### Second Part : Cleaning and Imputing Data For Predicting Player's Value

In [5]:
players = pd.read_csv(r'player_data_selected.csv')
players

C:\Users\shubhanshu gupta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (62,63,73,74,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date,Contract Remaining,Category
0,0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
1,1,2008,Kaká,25,90,91,Milan,138449,182.9,161,Right,90,CAM,1,0,0,0,334,87,90,64,93,NaN,353,91,NaN,80,90,92,271,93,89,NaN,89,NaN,330,86,NaN,78,73,93,288,38,87,86,NaN,77,NaN,51,22,29,NaN,70,4,22,90,22,22,1697,470,4,1,NaN,NaN,4,91,89,89,91,42,68,1,CAM,NaN,NaN,NaN,2009,2009.0,1,Midfielder
2,2,2008,Diego,22,88,90,SV Werder Bremen,136138,172.7,161,Right,88,CAM,2,0,0,0,316,84,85,55,92,NaN,350,90,NaN,83,84,93,252,86,83,NaN,83,NaN,301,82,NaN,78,62,79,298,64,80,83,NaN,71,NaN,66,25,41,NaN,75,9,22,84,22,22,1658,455,4,1,NaN,NaN,3,84,82,87,90,45,67,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
3,3,2008,S. Nasri,20,85,90,Olympique de Marseille,165239,177.8,165,Right,85,CAM,5,0,0,0,310,77,76,67,90,NaN,334,88,NaN,75,84,87,247,82,81,NaN,84,NaN,299,76,NaN,75,69,79,300,64,89,80,NaN,67,NaN,84,37,47,NaN,75,12,21,84,21,21,1649,453,4,1,NaN,NaN,3,81,77,84,87,54,70,1,CAM,NaN,NaN,NaN,2009,2009.0,1,Midfielder
4,4,2008,Deco,29,85,90,FC Barcelona,53769,177.8,161,Right,85,CAM,5,0,0,0,305,82,74,58,91,NaN,342,82,NaN,84,86,90,254,86,81,NaN,87,NaN,314,74,NaN,82,72,86,332,71,89,84,NaN,88,NaN,47,25,22,NaN,75,12,21,86,21,21,1669,449,4,1,NaN,NaN,4,83,78,87,86,40,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,233658,2020,L. Klassen,19,58,75,TSV 1860 München,244181,172.7,152,Left,60,LWB,17,250000,500,519000,225,59,39,39,59,29.0,241,59,39.0,31,49,63,349,75,71,73.0,49,81.0,262,40,72.0,67,51,32,220,57,49,41,41.0,32,52.0,156,50,51,55.0,46,10,9,6,10,11,1499,331,3,2,High,Medium,1,73,37,52,62,50,57,2,LB,LM,NaN,NaN,2018,2020.0,0,Defender
233659,233659,2020,J. King,18,58,75,Sydney FC,246005,177.8,150,Left,60,LWB,17,240000,500,450000,202,52,27,39,56,28.0,217,56,26.0,30,50,55,331,76,71,65.0,50,69.0,255,29,65.0,76,56,29,230,52,54,55,38.0,31,41.0,163,55,56,52.0,50,8,10,11,11,10,1448,321,3,2,Medium,Medium,1,73,29,48,57,53,61,1,LB,NaN,NaN,NaN,2018,2020.0,0,Defender
233660,233660,2020,L. Obexer,22,57,68,FC Lugano,233502,177.8,159,Left,58,LWB,11,160000,1000,0,218,54,36,50,47,31.0,214,59,36.0,30,39,50,323,68,66,58.0,55,76.0,250,34,62.0,65,60,29,238,55,52,51,40.0,40,41.0,166,52,58,56.0,45,14,12,8,5,6,1454,317,3,2,Medium,Medium,1,67,35,44,57,54,60,2,LWB,LB,NaN,NaN,"Jun 30, 2020 On Loan",2020.0,0,Defender
233661,233661,2020,R. Donelon,23,57,62,Sligo Rovers,223091,170.2,157,Left,59,LWB,5,120000,500,195000,197,58,22,42,56,19.0,291,57,59.0,66,54,55,321,63,66,68.0,45,79.0,291,45,66.0,74,56,50,246,49,52,58,52.0,35,57.0,164,55,53,56.0,63,14,14,12,9,14,1573,327,2,2,High,Medium,1,65,35,56,58,53,60,3,LB,LWB,LM,NaN,2013,2020.0,0,Defender


In [6]:
players[players['Name'] == 'Ronaldinho']

,Unnamed: 0,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date,Contract Remaining,Category
0,0,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
416,416,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
419,419,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
422,422,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
425,425,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
428,428,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
431,431,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
434,434,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
437,437,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder
440,440,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010,2010.0,2,Midfielder


In [7]:
# Initial years of the website have same record for one player mentioned several times so just keeping the last one
players.drop_duplicates(subset=["Year","Name"],keep="last", inplace=True)

In [8]:
players[players['Name'] == 'Ronaldinho']

,Unnamed: 0,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date,Contract Remaining,Category
671,671,2008,Ronaldinho,27,91,93,FC Barcelona,28130,180.3,179,Right,91,CAM,2,0,0,0,329,83,90,66,90,NaN,372,97,NaN,93,85,97,271,89,91,NaN,91,NaN,333,84,NaN,80,76,93,317,66,81,87,NaN,83,NaN,50,22,28,NaN,78,12,22,85,22,22,1750,478,4,1,NaN,NaN,5,90,89,87,96,41,75,1,CAM,NaN,NaN,NaN,2010.0,2010.0,2,Midfielder
1788,1788,2011,Ronaldinho,30,82,87,Flamengo,28130,182.9,176,Right,82,CAM,5,0,0,0,379,82,79,51,85,82.0,442,91,91.0,85,82,93,370,69,70,80.0,80,71.0,340,79,52.0,61,65,83,298,30,25,76,84.0,83,NaN,59,12,28,19.0,54,7,14,13,6,14,1942,403,4,5,NaN,NaN,5,70,80,84,89,24,56,4,CAM,LF,LW,CF,2011,2014.0,3,Midfielder
2547,2547,2012,Ronaldinho,31,84,84,Flamengo,28130,182.9,176,Right,83,CAM,0,13000000,110000,0,376,80,77,51,86,82.0,439,90,91.0,85,80,93,357,66,61,74.0,77,79.0,315,73,32.0,53,76,81,330,60,25,77,85.0,83,NaN,59,12,28,19.0,54,7,14,13,6,14,1930,402,4,5,Medium,Low,3,63,78,84,88,24,65,4,CAM,LW,CF,ST,2011,2014.0,2,Midfielder
4469,4469,2013,Ronaldinho,32,82,82,Atlético Mineiro,28130,182.9,176,Right,80,CAM,0,85000000,80000,0,375,84,70,51,88,82.0,442,87,90.0,88,86,91,336,58,51,75.0,74,78.0,329,78,64.0,41,74,72,326,60,25,69,86.0,86,NaN,59,12,28,19.0,54,7,14,13,6,14,1921,386,4,5,Medium,Low,4,54,73,87,86,24,62,1,CAM,NaN,NaN,NaN,2012,2013.0,0,Midfielder
6469,6469,2014,Ronaldinho,33,81,81,Atlético Mineiro,28130,182.9,176,Right,79,CAM,0,5000000,80000,0,373,84,70,51,86,82.0,439,86,89.0,87,86,91,326,56,51,75.0,72,72.0,326,75,64.0,41,74,72,323,60,25,66,86.0,86,NaN,78,25,28,25.0,54,7,14,13,6,14,1919,388,4,5,Medium,Low,4,53,73,86,85,29,62,1,CAM,NaN,NaN,NaN,2012,2014.0,0,Midfielder
8079,8079,2015,Ronaldinho,34,78,78,Querétaro,28130,182.9,168,Right,77,CAM,0,41000000,45000,0,367,84,69,50,82,82.0,430,86,86.0,87,81,90,313,50,48,73.0,70,72.0,314,74,55.0,41,72,72,319,60,25,66,82.0,86,NaN,78,25,28,25.0,54,7,14,13,6,14,1875,377,4,5,Low,Low,3,49,72,83,84,28,61,2,CAM,ST,NaN,NaN,2014,2016.0,1,Midfielder
22977,22977,2009,Ronaldinho,28,85,93,Milan,28130,180.3,176,Right,88,CF,8,0,0,0,323,85,89,59,90,NaN,366,97,NaN,93,80,96,240,74,78,NaN,88,NaN,300,81,NaN,69,61,89,267,31,62,85,NaN,89,NaN,50,22,28,NaN,75,9,22,80,22,22,1621,438,4,1,NaN,NaN,5,76,87,87,95,36,57,1,LF,NaN,NaN,NaN,2011,2011.0,2,Forward
27664,27664,2010,Ronaldinho,29,84,91,Milan,28130,182.9,176,Right,87,CF,7,0,0,0,328,85,85,68,90,NaN,355,92,NaN,87,80,96,237,76,77,NaN,84,NaN,295,84,NaN,62,64,85,267,30,62,85,NaN,90,NaN,50,22,28,NaN,75,9,22,80,22,22,1607,434,4,1,NaN,NaN,5,77,85,86,92,37,57,1,LF,NaN,NaN,NaN,2012,2012.0,2,Forward


In [11]:
## Value is the target variable but it is zero in some records so just keeping records for which it is not zero
players = players[(players['Value'] != 0)]
players

,Unnamed: 0,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Position 2,Position 3,Position 4,Contract Start Date,Contract End Date,Contract Remaining,Category
2543,2543,2012,M. Götze,18,84,91,Borussia Dortmund,192318,175.3,141,Right,83,CAM,7,41000000,120000,0,352,70,72,48,86,76.0,408,86,83.0,74,78,87,436,84,83,88.0,88,93.0,342,68,63.0,84,66,61,338,45,54,85,86.0,68,NaN,76,16,29,31.0,49,14,7,12,6,10,2001,419,4,4,Medium,Medium,3,83,70,81,87,32,66,3,CAM,LM,RM,NaN,2009,2014.0,2,Midfielder
2544,2544,2012,Hernanes,26,84,87,Lazio,180432,180.3,168,Right,83,CAM,3,23000000,110000,0,384,79,78,60,86,81.0,425,84,83.0,84,86,88,360,65,71,76.0,84,64.0,385,85,69.0,78,68,85,388,70,68,80,86.0,84,NaN,191,54,69,68.0,53,10,12,14,8,9,2186,450,5,4,High,Medium,3,68,81,84,83,63,71,4,CAM,CM,CDM,RM,2010,2015.0,3,Midfielder
2545,2545,2012,M. Hamšik,23,84,88,Napoli,171877,182.9,161,Right,83,CAM,4,24000000,110000,0,403,80,80,76,86,81.0,394,76,81.0,74,78,85,375,77,76,79.0,83,60.0,381,80,74.0,78,67,82,376,66,60,82,86.0,82,NaN,191,62,66,63.0,51,9,7,5,15,15,2171,453,4,3,High,Medium,3,76,81,83,79,64,70,3,CAM,CF,CM,NaN,2007,2016.0,4,Midfielder
2547,2547,2012,Ronaldinho,31,84,84,Flamengo,28130,182.9,176,Right,83,CAM,0,13000000,110000,0,376,80,77,51,86,82.0,439,90,91.0,85,80,93,357,66,61,74.0,77,79.0,315,73,32.0,53,76,81,330,60,25,77,85.0,83,NaN,59,12,28,19.0,54,7,14,13,6,14,1930,402,4,5,Medium,Low,3,63,78,84,88,24,65,4,CAM,LW,CF,ST,2011,2014.0,2,Midfielder
2548,2548,2012,Danny,27,84,87,Zenit St. Petersburg,20800,177.8,154,Right,83,CAM,3,23000000,130000,0,347,77,69,44,86,71.0,409,86,78.0,79,80,86,409,81,82,84.0,81,81.0,331,77,72.0,71,40,71,369,84,41,81,86.0,77,NaN,88,23,33,32.0,40,7,11,6,11,5,1993,413,4,5,Medium,Low,3,82,72,83,85,33,58,4,CAM,LM,CF,CM,2008,2015.0,3,Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233658,233658,2020,L. Klassen,19,58,75,TSV 1860 München,244181,172.7,152,Left,60,LWB,17,250000,500,519000,225,59,39,39,59,29.0,241,59,39.0,31,49,63,349,75,71,73.0,49,81.0,262,40,72.0,67,51,32,220,57,49,41,41.0,32,52.0,156,50,51,55.0,46,10,9,6,10,11,1499,331,3,2,High,Medium,1,73,37,52,62,50,57,2,LB,LM,NaN,NaN,2018,2020.0,0,Defender
233659,233659,2020,J. King,18,58,75,Sydney FC,246005,177.8,150,Left,60,LWB,17,240000,500,450000,202,52,27,39,56,28.0,217,56,26.0,30,50,55,331,76,71,65.0,50,69.0,255,29,65.0,76,56,29,230,52,54,55,38.0,31,41.0,163,55,56,52.0,50,8,10,11,11,10,1448,321,3,2,Medium,Medium,1,73,29,48,57,53,61,1,LB,NaN,NaN,NaN,2018,2020.0,0,Defender
233660,233660,2020,L. Obexer,22,57,68,FC Lugano,233502,177.8,159,Left,58,LWB,11,160000,1000,0,218,54,36,50,47,31.0,214,59,36.0,30,39,50,323,68,66,58.0,55,76.0,250,34,62.0,65,60,29,238,55,52,51,40.0,40,41.0,166,52,58,56.0,45,14,12,8,5,6,1454,317,3,2,Medium,Medium,1,67,35,44,57,54,60,2,LWB,LB,NaN,NaN,"Jun 30, 2020 On Loan",2020.0,0,Defender
233661,233661,2020,R. Donelon,23,57,62,Sligo Rovers,223091,170.2,157,Left,59,LWB,5,120000,500,195000,197,58,22,42,56,19.0,291,57,59.0,66,54,55,321,63,66,68.0,45,79.0,291,45,66.0,74,56,50,246,49,52,58,52.0,35,

In [18]:
# We see that all the data before 2012 is removed
players['Year'].value_counts()

2019    16757
2020    16699
2018    16698
2017    16314
2016    15746
2015    15173
2014    15142
2013    13753
2012    13259
Name: Year, dtype: int64

In [ ]:
players.drop(labels = ['Unnamed: 0'],axis=1,inplace = True)

In [19]:
# Keeping data for 2019 and 2020 as separate for testing
players_train = players[players['Year'].isin(['2012','2013','2014','2015','2016','2017','2018'])]
players_test = players[players['Year'].isin(['2019','2020'])]
players_test.to_csv(f'test_data.csv', encoding='utf-8-sig')

### Steps to clean train data. Will then be followed for test data

In [20]:
missing_Values_Columns = players_train.isnull().sum(axis = 0)
missing_Values_Percentage = missing_Values_Columns * 100/players_train.shape[0]
missing_Value_df = missing_Values_Percentage.to_frame()
missing_Value_df[missing_Value_df[0] > 0]

,0
Composure,68.881557
Position 2,48.978649
Position 3,82.085120
Position 4,95.658199


In [21]:
# Removing the features
players_train.drop(labels = ['Composure','Position 2','Position 3','Position 4'],axis=1,inplace = True)

In [22]:
corr = players.corr(method ='pearson')
# Create correlation matrix
corr_matrix = corr.abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

['Best Overall',
 'Marking',
 'Standing Tackle',
 'Sliding Tackle',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes']

In [23]:
players_train

,Year,Name,Age,Rating,Potential,Team,ID,Height,Weight,Foot,Best Overall,Best Position,Growth,Value,Wage,Release Clause,Total Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Total Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Total Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Total Power,Short Power,Jumping,Stamina,Strength,Long Shots,Total Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Total Defending,Marking,Standing Tackle,Sliding Tackle,Total GK,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,Weak Foot,Skill Moves,Attacking Work Rate,Defensive Work Rate,International Reputation,PAC,SHO,PAS,DRI,DEF,PHY,TotalPositions,Position 1,Contract Start Date,Contract End Date,Contract Remaining,Category
2543,2012,M. Götze,18,84,91,Borussia Dortmund,192318,175.3,141,Right,83,CAM,7,41000000,120000,0,352,70,72,48,86,76.0,408,86,83.0,74,78,87,436,84,83,88.0,88,93.0,342,68,63.0,84,66,61,338,45,54,85,86.0,68,76,16,29,31.0,49,14,7,12,6,10,2001,419,4,4,Medium,Medium,3,83,70,81,87,32,66,3,CAM,2009,2014.0,2,Midfielder
2544,2012,Hernanes,26,84,87,Lazio,180432,180.3,168,Right,83,CAM,3,23000000,110000,0,384,79,78,60,86,81.0,425,84,83.0,84,86,88,360,65,71,76.0,84,64.0,385,85,69.0,78,68,85,388,70,68,80,86.0,84,191,54,69,68.0,53,10,12,14,8,9,2186,450,5,4,High,Medium,3,68,81,84,83,63,71,4,CAM,2010,2015.0,3,Midfielder
2545,2012,M. Hamšik,23,84,88,Napoli,171877,182.9,161,Right,83,CAM,4,24000000,110000,0,403,80,80,76,86,81.0,394,76,81.0,74,78,85,375,77,76,79.0,83,60.0,381,80,74.0,78,67,82,376,66,60,82,86.0,82,191,62,66,63.0,51,9,7,5,15,15,2171,453,4,3,High,Medium,3,76,81,83,79,64,70,3,CAM,2007,2016.0,4,Midfielder
2547,2012,Ronaldinho,31,84,84,Flamengo,28130,182.9,176,Right,83,CAM,0,13000000,110000,0,376,80,77,51,86,82.0,439,90,91.0,85,80,93,357,66,61,74.0,77,79.0,315,73,32.0,53,76,81,330,60,25,77,85.0,83,59,12,28,19.0,54,7,14,13,6,14,1930,402,4,5,Medium,Low,3,63,78,84,88,24,65,4,CAM,2011,2014.0,2,Midfielder
2548,2012,Danny,27,84,87,Zenit St. Petersburg,20800,177.8,154,Right,83,CAM,3,23000000,130000,0,347,77,69,44,86,71.0,409,86,78.0,79,80,86,409,81,82,84.0,81,81.0,331,77,72.0,71,40,71,369,84,41,81,86.0,77,88,23,33,32.0,40,7,11,6,11,5,1993,413,4,5,Medium,Low,3,82,72,83,85,33,58,4,CAM,2008,2015.0,3,Midfielder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233043,2018,M. Johannsen,25,54,57,FC Helsingør,239155,185.4,183,Left,53,LWB,3,70000,2000,93000,204,59,34,37,51,23.0,203,47,45.0,29,42,40,308,68,58,58.0,53,71.0,240,28,63.0,56,67,26,211,53,45,39,39.0,35,165,52,59,54.0,52,12,14,9,8,9,1383,301,2,2,Medium,Medium,1,63,31,47,48,51,61,2,LWB,2017,2019.0,1,Defender
233044,2018,G. Harkin,29,54,54,Finn Harps,231593,180.3,176,Left,53,LWB,0,50000,500,78000,223,57,42,44,54,26.0,251,51,55.0,44,48,53,236,48,46,48.0,48,46.0,234,41,50.0,61,54,28,224,58,52,33,46.0,35,162,54,54,54.0,50,11,11,15,6,7,1380,296,2,2,Low,Medium,1,47,37,52,51,53,56,3,LB,2014,2020.0,2,Defender
233045,2018,A. Kawai,17,54,67,Sanfrecce Hiroshima,244651,177.8,143,Left,53,LWB,13,100000,1000,188000,205,62,35,40,43,25.0,234,52,56.0,26,47,53,319,66,62,64.0,53,74.0,215,30,60.0,57,41,27,220,56,47,46,37.0,34,151,41,58,52.0,38,5,5,11,8,9,1382,294,3,2,Medium,Medium,1,64,32,46,55,48,49,1,LB,2018,2020.0,2,Defender
233046,2018,M. O'Brien,19,51,58,Drogheda United,230710,175.3,157,Left,50,LWB,7,40000,500,70000,201,46,35,41,46,33.0,224,47,45.0,43,43,46,294,66,66,58.0,46,58.0,245,42,55.0,69,42,37,210,46,46,42,41.0,35,136,45,47,44.0,56,11,15,12,11,7,1366,290,2,2,Medium,Medium,1,66,37,44,48,45,50,2,LB,2015,2020.0,2,Defender


In [25]:
# Changinng object columns to category columns

changeTypeToCategory = ['Team','Foot','Best Position','Position 1','Category','Defensive Work Rate','Attacking Work Rate']
for col in changeTypeToCategory:
    players_train[col] = players_train[col].astype('category')

In [ ]:
names = players_train['Name']
players_train.drop(['Year','Name','Team',''])